<a href="https://colab.research.google.com/github/dhanifawzan/TA/blob/branccontohmain/cobarekomenlagi.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

combined_movies_data=pd.read_csv('musikkukerasboss.csv', sep=r';')
combined_movies_data.head()

,index,user_id,menu_id,rating
0,1,U00002,L00010,4
1,2,U00002,L00011,3
2,3,U00002,L00012,5
3,4,U00002,L00013,4
4,5,U00002,L00014,5


In [2]:
combined_movies_data=combined_movies_data.drop('index',axis=1)

In [3]:
# my user_id is the 1001
my_ratings = pd.read_csv('yanginibosscape.csv')
my_ratings 


,user_id;menu_id;rating
0,U0004;L00010;5
1,U0004;L00011;2
2,U0004;L00012;1
3,U0004;L00013;2
4,U0004;L00014;1


In [4]:
combined_movies_data = pd.concat([combined_movies_data, my_ratings], axis=0)

# rename the columns to userID, itemID and rating
combined_movies_data.columns = ['user_id', 'menu_id', 'rating','reviews']

# use the transform method group by userID and count to keep the movies with more than 25 reviews

combined_movies_data['reviews'] = combined_movies_data.groupby(['menu_id'])['rating'].transform('count')

combined_movies_data= combined_movies_data[combined_movies_data.reviews>10][['user_id', 'menu_id', 'rating']]

In [5]:

pip install scipy==1.0.0
pip install surprise

     |████████████████████████████████| 15.2 MB 175 kB/s 
  ERROR: Failed building wheel for scipy
  Running setup.py clean for scipy
  ERROR: Failed cleaning build dir for scipy
Failed to build scipy
  Attempting uninstall: scipy
    Found existing installation: scipy 1.4.1
    Uninstalling scipy-1.4.1:
      Successfully uninstalled scipy-1.4.1
    Running setup.py install for scipy ... error
  Rolling back uninstall of scipy
  Moving to /usr/local/lib/python3.7/dist-packages/scipy-1.4.1.dist-info/
   from /usr/local/lib/python3.7/dist-packages/~cipy-1.4.1.dist-info
  Moving to /usr/local/lib/python3.7/dist-packages/scipy/
   from /usr/local/lib/python3.7/dist-packages/~cipy
ERROR: Command errored out with exit status 1: /usr/bin/python3 -u -c 'import io, os, sys, setuptools, tokenize; sys.argv[0] = '"'"'/tmp/pip-install-h0ma1k2w/scipy_5fec738223094989ab03450da78db12d/setup.py'"'"'; __file__='"'"'/tmp/pip-install-h0ma1k2w/scipy_5fec738223094989ab03450da78db12d/setup.py'"'"';f = getat

In [6]:
from surprise import NMF, SVD, SVDpp, KNNBasic, KNNWithMeans, KNNWithZScore, CoClustering
from surprise.model_selection import cross_validate
from surprise import Reader, Dataset

In [7]:
# A reader is still needed but only the rating_scale param is requiered.
reader = Reader(rating_scale=(1, 5))
data = Dataset.load_from_df(combined_movies_data, reader)

In [15]:
# get the list of the movie ids
unique_ids = combined_movies_data['menu_id'].unique()

# get the list of the ids that the userid 1001 has rated
iids1001 = combined_movies_data.loc[combined_movies_data['user_id']=='U0004', 'menu_id']

# remove the rated movies for the recommendations
movies_to_predict = np.setdiff1d(unique_ids,iids1001)

In [16]:
algo = SVD()
algo.fit(data.build_full_trainset())

my_recs = []
for iid in movies_to_predict:
    my_recs.append((iid, algo.predict(uid='U0004',iid=iid).est))
    
pd.DataFrame(my_recs, columns=['iid', 'predictions']).sort_values('predictions', ascending=False).head(10)

,iid,predictions
106,L00116,3.451933
225,L00235,3.263547
162,L00172,3.262803
61,L00071,3.246709
35,L00045,3.221698
238,L00248,3.191297
229,L00239,3.169550
206,L00216,3.166520
77,L00087,3.132680
237,L00247,3.113998


In [17]:
cv = []
# Iterate over all recommender system algorithms
for recsys in [NMF(), SVD(), SVDpp(), KNNWithZScore(), CoClustering()]:
    # Perform cross validation
    tmp = cross_validate(recsys, data, measures=['RMSE'], cv=3, verbose=False)
    cv.append((str(recsys).split(' ')[0].split('.')[-1], tmp['test_rmse'].mean()))

pd.DataFrame(cv, columns=['RecSys', 'RMSE'])

Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.
Computing the msd similarity matrix...
Done computing similarity matrix.


,RecSys,RMSE
0,NMF,2.031379
1,SVD,1.786165
2,SVDpp,1.861702
3,KNNWithZScore,1.781762
4,CoClustering,1.855325


In [18]:
pip install uvicorn

     |████████████████████████████████| 50 kB 6.1 MB/s 
     |████████████████████████████████| 54 kB 3.7 MB/s 


In [19]:
pip install fastapi

     |████████████████████████████████| 51 kB 393 kB/s 
     |████████████████████████████████| 60 kB 7.5 MB/s 
     |████████████████████████████████| 10.1 MB 42.7 MB/s 
